In [113]:
import pandas as pd
import numpy as np

In [112]:
# Fetch the dataframes of known cancertypes (they are sorted by cancertype)
rna = pd.read_csv("/Users/bram/Desktop/CSE3000/data/clamped_3modal_RNA_3000MAD_cancertypeknown.csv", index_col=0)
gcn = pd.read_csv("/Users/bram/Desktop/CSE3000/data/clamped_3modal_GCN_3000MAD_cancertypeknown.csv", index_col=0)
dna = pd.read_csv("/Users/bram/Desktop/CSE3000/data/clamped_3modal_DNA_3000MAD_cancertypeknown.csv", index_col=0)

In [111]:
# Fetch the cancertype labels (same order as dataframes)
labels = np.load("/Users/bram/Desktop/CSE3000/data/cancertypes_samples.npy")

In [52]:
# # Shuffle labels and dataframes for training, but keep the same order of samples and labels
# shuffler = np.random.permutation(len(labels))

# labels_shuffle = labels[shuffler]
# rna_shuffle = rna.iloc[shuffler]
# gcn_shuffle = gcn.iloc[shuffler]
# dna_shuffle = dna.iloc[shuffler]

# np.save("/Users/bram/Desktop/CSE3000/data/shuffle_cancertypes_samples.npy", labels_shuffle)
# rna_shuffle.to_csv("/Users/bram/Desktop/CSE3000/data/shuffle_clamped_3modal_RNA_3000MAD_cancertypeknown.csv")
# gcn_shuffle.to_csv("/Users/bram/Desktop/CSE3000/data/shuffle_clamped_3modal_GCN_3000MAD_cancertypeknown.csv")
# dna_shuffle.to_csv("/Users/bram/Desktop/CSE3000/data/shuffle_clamped_3modal_DNA_3000MAD_cancertypeknown.csv")

In [114]:
# Also create a dataset with only 3 cancertypes
# only use 3 most common cancer types (BRCA : 766, KIRC : 306, LUAD : 449)
brca = np.where(labels == 'BRCA')[0]
kirc = np.where(labels == 'KIRC')[0]
luad = np.where(labels == 'LUAD')[0]

# Create list of the three cancer type indices
cancer3 = [brca, kirc, luad]
cancer3

[array([ 480,  481,  482,  483,  484,  485,  486,  487,  488,  489,  490,
         491,  492,  493,  494,  495,  496,  497,  498,  499,  500,  501,
         502,  503,  504,  505,  506,  507,  508,  509,  510,  511,  512,
         513,  514,  515,  516,  517,  518,  519,  520,  521,  522,  523,
         524,  525,  526,  527,  528,  529,  530,  531,  532,  533,  534,
         535,  536,  537,  538,  539,  540,  541,  542,  543,  544,  545,
         546,  547,  548,  549,  550,  551,  552,  553,  554,  555,  556,
         557,  558,  559,  560,  561,  562,  563,  564,  565,  566,  567,
         568,  569,  570,  571,  572,  573,  574,  575,  576,  577,  578,
         579,  580,  581,  582,  583,  584,  585,  586,  587,  588,  589,
         590,  591,  592,  593,  594,  595,  596,  597,  598,  599,  600,
         601,  602,  603,  604,  605,  606,  607,  608,  609,  610,  611,
         612,  613,  614,  615,  616,  617,  618,  619,  620,  621,  622,
         623,  624,  625,  626,  627, 

In [115]:
# Now get the indices 70-10-20 split, where every split has the same relative representation of cancertypes
TRAINING_DATA_SPLIT = 0.7
VALIDATION_DATA_SPLIT = 0.1
PREDICT_DATA_SPLIT = 0.2

all_training_ids = []
all_validation_ids = []
all_predict_ids = []

all_training_labels = []
all_validation_labels = []
all_predict_labels = []

label_names = ["BRCA", "KIRC", "LUAD"]

for cancer_type in range(len(cancer3)):

    nr_of_samples = len(cancer3[cancer_type])
    nr_of_training_samples = int(TRAINING_DATA_SPLIT * nr_of_samples)
    nr_of_validation_samples = int(VALIDATION_DATA_SPLIT * nr_of_samples)

    # Random ordering of all sample id's
    random_sample_indices = np.random.choice(a=cancer3[cancer_type], size=nr_of_samples, replace=False)

    # Split into three sets of sizes
    # [:nr_of_training_samples], [nr_of_training_samples:nr_of_validation_samples], [:nr_of_predict_samples]
    sets = np.split(random_sample_indices,
                    [nr_of_training_samples, (nr_of_training_samples + nr_of_validation_samples)])

    training_ids = sets[0]
    validation_ids = sets[1]
    predict_ids = sets[2]

    print(len(training_ids), len(validation_ids), len(predict_ids))
    all_training_ids.extend(training_ids)
    all_validation_ids.extend(validation_ids)
    all_predict_ids.extend(predict_ids)
    
    all_training_labels.extend([label_names[cancer_type]] * len(training_ids))
    all_validation_labels.extend([label_names[cancer_type]] * len(validation_ids))
    all_predict_labels.extend([label_names[cancer_type]] * len(predict_ids))


print(len(all_training_ids), len(all_validation_ids), len(all_predict_ids))
len(all_predict_labels)

536 76 154
214 30 62
314 44 91
1064 150 307


307

In [116]:
rna_training_3types = rna.iloc[all_training_ids]
gcn_training_3types = gcn.iloc[all_training_ids]
dna_training_3types = dna.iloc[all_training_ids]

rna_validation_3types = rna.iloc[all_validation_ids]
gcn_validation_3types = gcn.iloc[all_validation_ids]
dna_validation_3types = dna.iloc[all_validation_ids]

rna_predict_3types = rna.iloc[all_predict_ids]
gcn_predict_3types = gcn.iloc[all_predict_ids]
dna_predict_3types = dna.iloc[all_predict_ids]

In [129]:
# We now have data splits for each modality, and also the labels for each split
# For true randomness, we need to shuffle them all

# Shuffle labels and dataframes for training, but keep the same order of samples and labels
training_shuffler = np.random.permutation(len(all_training_labels))

training_labels_shuffle = np.take(all_training_labels, training_shuffler)
rna_training_shuffle = rna_training_3types.iloc[training_shuffler]
gcn_training_shuffle = gcn_training_3types.iloc[training_shuffler]
dna_training_shuffle = dna_training_3types.iloc[training_shuffler]

np.save("/Users/bram/Desktop/CSE3000/data/3types/training_3types.npy", training_labels_shuffle)
rna_training_shuffle.to_csv("/Users/bram/Desktop/CSE3000/data/3types/rna_preprocess_3types_training.csv")
gcn_training_shuffle.to_csv("/Users/bram/Desktop/CSE3000/data/3types/gcn_preprocess_3types_training.csv")
dna_training_shuffle.to_csv("/Users/bram/Desktop/CSE3000/data/3types/dna_preprocess_3types_training.csv")

In [130]:
# We now have data splits for each modality, and also the labels for each split
# For true randomness, we need to shuffle them all

# Shuffle labels and dataframes for validation, but keep the same order of samples and labels
validation_shuffler = np.random.permutation(len(all_validation_labels))

validation_labels_shuffle = np.take(all_validation_labels, validation_shuffler)
rna_validation_shuffle = rna_validation_3types.iloc[validation_shuffler]
gcn_validation_shuffle = gcn_validation_3types.iloc[validation_shuffler]
dna_validation_shuffle = dna_validation_3types.iloc[validation_shuffler]

np.save("/Users/bram/Desktop/CSE3000/data/3types/validation_3types.npy", validation_labels_shuffle)
rna_validation_shuffle.to_csv("/Users/bram/Desktop/CSE3000/data/3types/rna_preprocess_3types_validation.csv")
gcn_validation_shuffle.to_csv("/Users/bram/Desktop/CSE3000/data/3types/gcn_preprocess_3types_validation.csv")
dna_validation_shuffle.to_csv("/Users/bram/Desktop/CSE3000/data/3types/dna_preprocess_3types_validation.csv")

In [131]:
# We now have data splits for each modality, and also the labels for each split
# For true randomness, we need to shuffle them all

# Shuffle labels and dataframes for predict, but keep the same order of samples and labels
predict_shuffler = np.random.permutation(len(all_predict_labels))

predict_labels_shuffle = np.take(all_predict_labels, predict_shuffler)
rna_predict_shuffle = rna_predict_3types.iloc[predict_shuffler]
gcn_predict_shuffle = gcn_predict_3types.iloc[predict_shuffler]
dna_predict_shuffle = dna_predict_3types.iloc[predict_shuffler]

np.save("/Users/bram/Desktop/CSE3000/data/3types/predict_3types.npy", predict_labels_shuffle)
rna_predict_shuffle.to_csv("/Users/bram/Desktop/CSE3000/data/3types/rna_preprocess_3types_predict.csv")
gcn_predict_shuffle.to_csv("/Users/bram/Desktop/CSE3000/data/3types/gcn_preprocess_3types_predict.csv")
dna_predict_shuffle.to_csv("/Users/bram/Desktop/CSE3000/data/3types/dna_preprocess_3types_predict.csv")
